In [1]:
import pandas as pd

from lib.functions_data import nca_name_mapping, earliest_record_check

In [2]:
import sys
from pathlib import Path
import os
cwd = os.getcwd()
parent = str(Path(cwd).parents[0])
sys.path.append(parent)

%matplotlib inline
%load_ext autoreload
%autoreload 2

To avoid pulling the full dataset down each time we re-run the notebook, a CSV of the cut-down dataset is saved for easier reloading.

In [3]:
#Checking for the cut of the full dataset and creating it if it doesn't exist:
try:
    dec = pd.read_csv(parent + '/data/jul22_euctr_extract.csv').drop('Unnamed: 0', axis=1)
except FileNotFoundError:
    cols = ['eudract_number_with_country', 'date_of_competent_authority_decision', 
            'clinical_trial_type', 'national_competent_authority', 'eudract_number',
            'date_on_which_this_record_was_first_entered_in_the_eudract_data',
            'trial_status', 'date_of_the_global_end_of_the_trial', 'trial_results']

    #You can use this URL if you want to download the full raw data
    data_link = 'https://www.dropbox.com/s/t41njroo12mx4y2/euctr_euctr_dump-2022-07-02-003206.csv.zip?dl=1'

    dec = pd.read_csv(data_link, compression='zip', low_memory=False, usecols=cols)
    dec.to_csv(parent + '/data/jul22_euctr_extract.csv')

In [4]:
#Quick look at the spread of trial statuses on the EUCTR without any exclusions
dec.trial_status.value_counts(dropna=False)

Completed                   63502
Ongoing                     27293
Prematurely Ended           12966
NaN                          2960
GB - no longer in EU/EEA     2797
Temporarily Halted            520
Restarted                     332
Not Authorised                 76
Prohibited by CA               37
Suspended by CA                23
Trial now transitioned          6
Name: trial_status, dtype: int64

The "date_of_competent_authority_decision" field has 2 nonsensical year values in which the correct value can reasonably be derived from context. We fix those below:

https://www.clinicaltrialsregister.eu/ctr-search/trial/2009-016759-22/DK

https://www.clinicaltrialsregister.eu/ctr-search/trial/2006-006947-30/FR

In [5]:
ind = dec[dec.date_of_competent_authority_decision.notnull() & 
          dec.date_of_competent_authority_decision.str.contains('210')].index
ind = ind.to_list()[0]

ind_2 = dec[dec.date_of_competent_authority_decision.notnull() & 
            dec.date_of_competent_authority_decision.str.contains('2077')].index
ind_2 = ind_2.to_list()[0]

dec.at[ind, 'date_of_competent_authority_decision'] = '2010-06-18'
dec.at[ind_2, 'date_of_competent_authority_decision'] = '2007-04-05'

In [6]:
#get rid of all protocols from non EU/EEA countries
dec_filt = dec[dec.clinical_trial_type != 'Outside EU/EEA'].reset_index(drop=True)

#lets see how many that is:
print(len(dec) - len(dec_filt))

1522


In [7]:
dec_ctas = dec[['eudract_number', 'eudract_number_with_country']].groupby('eudract_number').count()['eudract_number_with_country']

print(f'There are {len(dec_ctas)} registered trials and {dec_ctas.sum()} CTAs including non-EU/EEA CTAs')

There are 42369 registered trials and 110512 CTAs including non-EU/EEA CTAs


In [8]:
decf_ctas = dec_filt[['eudract_number', 'eudract_number_with_country']].groupby('eudract_number').count()['eudract_number_with_country']

print(f'There are {len(decf_ctas)} registered trials and {decf_ctas.sum()} CTAs excluding non-EU/EEA CTAs')

There are 41243 registered trials and 108990 CTAs excluding non-EU/EEA CTAs


In [9]:
#Making dates into dates and adding a column of just the "Year" for relevant dates

dec_filt['date_on_which_this_record_was_first_entered_in_the_eudract_data'] = pd.to_datetime(dec_filt['date_on_which_this_record_was_first_entered_in_the_eudract_data'])
dec_filt['entered_year'] = dec_filt['date_on_which_this_record_was_first_entered_in_the_eudract_data'].dt.year

dec_filt['date_of_competent_authority_decision'] = pd.to_datetime(dec_filt['date_of_competent_authority_decision'])
dec_filt['approved_year'] = dec_filt['date_of_competent_authority_decision'].dt.year

In [10]:
#Creating a copy of the original dataset we can mess with and
#renaming columns to better variable names

analysis_df = dec_filt.copy()

analysis_df.columns = ['eudract_number_country', 
                       'approved_date', 
                       'clinical_trial_type', 
                       'nca', 
                       'eudract_number', 
                       'date_entered', 
                       'trial_status', 
                       'completion_date', 
                       'trial_results', 
                       'entered_year', 
                       'approved_year']

#And update the NCA names to the more accurate recent names
analysis_df['nca'] = analysis_df['nca'].replace(nca_name_mapping)

In [11]:
#Table 1
analysis_df[['nca', 'eudract_number_country']].groupby('nca').count()

,eudract_number_country
nca,
Austria - BASG,4440
Belgium - FAMHP,6265
Bulgaria - BDA,2254
Croatia - MIZ,458
Cyprus - MoH-Ph.S,19
Czechia - SUKL,4687
Denmark - DKMA,4561
Estonia - SAM,1051
Finland - FIMEA,2702


In [12]:
#You can reproduce the data on the earliest registered protocol (For table 1) for each country by running 
#this function with the appropriate country abbreviation. For example, to get the date for Italy:

print(earliest_record_check(analysis_df, 'Italy - AIFA'))

#Uncomment this to get the date for all countries at once
#for abrev in country_abrevs.keys():
#    print(f'Country: {abrev}\nEarliest record date: {earliest_record_check(dec_filt, abrev)}')

2004-07-16


In [13]:
analysis_df.head()

,eudract_number_country,approved_date,clinical_trial_type,nca,eudract_number,date_entered,trial_status,completion_date,trial_results,entered_year,approved_year
0,2020-004353-80-FI,2021-01-04,EEA CTA,Finland - FIMEA,2020-004353-80,2020-12-07,Ongoing,NaN,NaN,2020,2021.0
1,2008-006344-19-CZ,2009-06-15,EEA CTA,Czechia - SUKL,2008-006344-19,2009-05-05,Completed,2011-11-20,View results,2009,2009.0
2,2011-004581-14-DE,2012-04-16,EEA CTA,Germany - BfArM,2011-004581-14,2012-03-29,Completed,NaN,View results,2012,2012.0
3,2010-021638-72-IT,2011-06-21,EEA CTA,Italy - AIFA,2010-021638-72,2012-01-23,Completed,2015-07-15,View results,2012,2011.0
4,2019-002880-82-GB,2020-05-05,EEA CTA,UK - MHRA,2019-002880-82,2020-06-02,GB - no longer in EU/EEA,NaN,NaN,2020,2020.0


In [14]:
#To save the main analysis df for use by the other notebooks. Only need to run this once the first time.
#analysis_df.to_csv(parent + '/data/analysis_df_jul22.csv')

In [15]:
#lastly this is helpful to have the country names, both the original ones from the EUCTR and the 
#current ones, in ordered lists.
#We're going to make these available as variables to import in the lib file manually
#But you can see what they look like here:
ordered_countries_original = list(dec_filt.national_competent_authority.value_counts().index)
ordered_countries_new = list(analysis_df.nca.value_counts().index)

In [16]:
ordered_countries_new

['UK - MHRA',
 'Italy - AIFA',
 'Spain - AEMPS',
 'Germany - BfArM',
 'France - ANSM',
 'Netherlands - CCMO',
 'Belgium - FAMHP',
 'Hungary - OGYEI',
 'Czechia - SUKL',
 'Denmark - DKMA',
 'Austria - BASG',
 'Sweden - MPA',
 'Poland - URPL',
 'Germany - PEI',
 'Finland - FIMEA',
 'Bulgaria - BDA',
 'Greece - EOF',
 'Slovakia - SUKL',
 'Portugal - INFARMED',
 'Lithuania - VVKT',
 'Ireland - HPRA',
 'Latvia - ZVA',
 'Estonia - SAM',
 'Norway - NoMA',
 'Romania - ANMDM',
 'Croatia - MIZ',
 'Slovenia - JAZMP',
 'Iceland - IMA',
 'Cyprus - MoH-Ph.S',
 'Malta - ADM',
 'Luxembourg - Ministry of Health']

In [17]:
analysis_df.nca.value_counts()

UK - MHRA                          11041
Italy - AIFA                       10797
Spain - AEMPS                      10558
Germany - BfArM                     8898
France - ANSM                       6600
Netherlands - CCMO                  6394
Belgium - FAMHP                     6265
Hungary - OGYEI                     4909
Czechia - SUKL                      4687
Denmark - DKMA                      4561
Austria - BASG                      4440
Sweden - MPA                        4158
Poland - URPL                       3912
Germany - PEI                       3660
Finland - FIMEA                     2702
Bulgaria - BDA                      2254
Greece - EOF                        2032
Slovakia - SUKL                     1957
Portugal - INFARMED                 1767
Lithuania - VVKT                    1322
Ireland - HPRA                      1283
Latvia - ZVA                        1148
Estonia - SAM                       1051
Norway - NoMA                        888
Romania - ANMDM 

In [18]:
ordered_countries_original

['UK - MHRA',
 'Italy - Italian Medicines Agency',
 'Spain - AEMPS',
 'Germany - BfArM',
 'France - ANSM',
 'Netherlands - Competent Authority',
 'Belgium - FPS Health-DGM',
 'Hungary - National Institute of Pharmacy',
 'Czechia - SUKL',
 'Denmark - DHMA',
 'Austria - BASG',
 'Sweden - MPA',
 'Poland - Office for Medicinal Products',
 'Germany - PEI',
 'Finland - Fimea',
 'Bulgarian Drug Agency',
 'Greece - EOF',
 'Slovakia - SIDC (Slovak)',
 'Portugal - INFARMED',
 'Lithuania - SMCA',
 'Ireland - HPRA',
 'Latvia - SAM',
 'Estonia - SAM',
 'Norway - NOMA',
 'Romania - National Agency for Medicines and Medical Devices',
 'Croatia - MIZ',
 'Slovenia - JAZMP',
 'Iceland - IMCA',
 'Cyprus - MoH-Ph.S',
 'Malta - ADM',
 'Luxembourg - Ministry of Health',
 'CZECHIA - SKUL']